# [Issue #1357](https://github.com/cal-itp/data-analyses/issues/1357)
* <i>Since there was also another reported issue of a missing transit agency in #1254, I recommend cross referencing the complete GTFS Digest Agency List with the list of fixed-route agencies on the Statewide Transit Metrics dashboard.</i> 

In [1]:
import _operators_prep
import deploy_portfolio_yaml
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import portfolio_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
gtfs_digest_operators = _operators_prep.operators_schd_vp_rt()

In [4]:
gtfs_digest_operators.columns

Index(['caltrans_district', 'organization_name'], dtype='object')

In [21]:
gtfs_digest_operators.organization_name.nunique()

199

In [5]:
gtfs_digest_operators.organization_name = (
    gtfs_digest_operators.organization_name.str.lower().str.replace("-", " ")
)

In [6]:
metabase_fixed_route_agencies = "gs://calitp-analytics-data/data-analyses/gtfs_digest_other/list_of_fixed_route_transit_agencies_2025-01-23T18_30_01.768867Z.xlsx"

In [7]:
metabase_df = to_snakecase(pd.read_excel(metabase_fixed_route_agencies))

/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
metabase_df.name__and_link_to_agency_dashboard_.nunique()

198

In [10]:
metabase_df.name__and_link_to_agency_dashboard_ = (
    metabase_df.name__and_link_to_agency_dashboard_.str.lower().str.replace("-", " ")
)

In [11]:
len(metabase_df)

198

In [20]:
metabase_df.head(10)

,name__and_link_to_agency_dashboard_,organization_type,website
0,alameda contra costa transit district,Independent Agency,https://www.actransit.org/
1,amador regional transit system,Independent Agency,https://amadortransit.com/
2,amtrak,Federal Government,https://www.amtrak.com/home
3,anaheim transportation network,Independent Agency,https://rideart.org/
4,antelope valley transit authority,Independent Agency,https://www.avta.com/
5,basin transit,Independent Agency,Basin-Transit.com
6,butte county association of governments,MPO/RTPA,http://www.bcag.org/
7,calaveras transit agency,Joint Powers Agency,http://calaverasconnect.org/
8,capitol corridor joint powers authority,Independent Agency,https://www.capitolcorridor.org/
9,central contra costa transit authority,Joint Powers Agency,https://countyconnection.com/


In [14]:
gtfs_digest_operators.head()

,caltrans_district,organization_name
0,01 - Eureka,blue lake rancheria
1,01 - Eureka,city of arcata
2,01 - Eureka,city of eureka
3,01 - Eureka,curry public transit
4,01 - Eureka,humboldt transit authority


In [15]:
m1 = pd.merge(
    gtfs_digest_operators,
    metabase_df,
    left_on=["organization_name"],
    right_on=["name__and_link_to_agency_dashboard_"],
    how="outer",
    indicator=True,
)

In [16]:
m1._merge.value_counts()

both          185
left_only      14
right_only     13
Name: _merge, dtype: int64

In [18]:
m1.loc[m1._merge == "left_only"][['organization_name']]

,organization_name
0,blue lake rancheria
14,shasta county
28,north lake tahoe express
53,emeryville transportation management agency
56,mission bay transportation management agency
57,mountain view transportation management association
69,stanford university
70,"university of california, berkeley"
85,"university of california, santa cruz"
144,flixbus


In [19]:
m1.loc[m1._merge == "right_only"][['name__and_link_to_agency_dashboard_']]

,name__and_link_to_agency_dashboard_
199,city of alameda
200,city of beverly hills
201,city of carson
202,city of compton
203,city of laguna niguel
204,city of newport beach
205,city of san fernando
206,dumbarton bridge regional operations consortium
207,"golden gate bridge, highway and transportation district"
208,san bernardino county transportation authority
